In [1]:
from __future__ import print_function
import keras
import numpy as np
from keras.utils import Sequence
from sklearn.utils import shuffle
import re

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
class TweetSeqGenerator(Sequence):
  
    def __init__(self, data, labels, batch_size, normalize=False):
      self.data = data
      self.labels = labels
      if len(self.data) != len(self.labels):
        raise ValueError("Data and Labels have to be same size")
      self.batch_size = batch_size
      self.normalize = normalize
      
      # If normalize, then normalize each timeseries to sum to 1 on each keyword
      if self.normalize:
        for i in range(len(self.data)):
            # Normalizes the rows of the matrix to 1
            # Axis=0 should normalize the columns to 1
            self.data[i] = self.data[i]/np.max(self.data[i], axis=1)


    def __getitem__(self, index):
        """
        Go through each set of files and augment them as needed
        :param index:
        :return:
        """
        timeseries = self.data[index * self.batch_size:(index + 1) * self.batch_size]
        labels = self.labels[index * self.batch_size:(index + 1) * self.batch_size]

        return timeseries, labels

    def __len__(self):
        """
        Returns the length of the list of paths, as the number of events is not known
        :return:
        """
        return int(np.ceil(len(self.data) / float(self.batch_size)))

    def on_epoch_end(self):
            self.data = shuffle(self.data, self.labels)


In [3]:
def generate_timeseries(category, length, limit=None):
  """
  Generates timeseries from the tweets, and gets the next event to occur in the future
  """
  return NotImplementedError

In [4]:
def clean_tweet(tweet):
  """
  Cleans a tweet to just English characters, removing URLs, etc.
  What is given is just the tweet itself, not the metadata
  """
  tweet = re.sub(r"http\S+", "", tweet) # removes URLs
  tweet = re.sub(r"[^a-zA-Z0-9]+", ' ', tweet) # Removes non-alphanumeric chars
  tweet = tweet.lower() # Lowercases it
  
  return tweet
  
  

In [5]:
import keras
from keras.utils import Sequence
from sklearn.utils import shuffle
import re

from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
import numpy as np
import random
import sys
import io
import pandas as pd
import google.datalab.bigquery as bq
    
def getDATA():
    query_K2019tweets = """

    SELECT created_at,keywords FROM `hdc-politie-team-3.politie.tweets` WHERE iso_language_code='nl' and created_at BETWEEN "2019-04-26" AND "2019-04-28" ORDER BY created_at ASC

    """
    print(query_K2019tweets)
    # Transform your query to BigQuery object
    bq_object = bq.Query(query_K2019tweets)

    # Execute your query
    result = bq_object.execute().result()

    # Transform your output to a Pandas dataframe
    kings2019tweets_df = result.to_dataframe()
    
    return kings2019tweets_df
  

def getLabels():
  query_K2019tweets = """

  SELECT * FROM `hdc-politie-team-3.politie.meldkamer` WHERE start_incident BETWEEN "2019-04-26" AND "2019-04-28" ORDER BY start_incident ASC

  """
  print(query_K2019tweets)
  # Transform your query to BigQuery object
  bq_object = bq.Query(query_K2019tweets)

  # Execute your query
  result = bq_object.execute().result()

  # Transform your output to a Pandas dataframe
  kings2019mcl_df = result.to_dataframe()

  return kings2019mcl_df


def getkeywords():
    query_K2019keys = """

   SELECT distinct key FROM `hdc-politie-team-3.politie.tweets`,unnest(keywords) as key

    """
    # Transform your query to BigQuery object
    bq_object = bq.Query(query_K2019keys)
    
    # Execute your query
    result = bq_object.execute().result()

    # Transform your output to a Pandas dataframe
    kings2019keys_df = result.to_dataframe()
    aj=kings2019keys_df.values.tolist()
    newa=[]
    for i in range(len(aj)):
        newa.append(aj[i][0])
    
    nexta=sorted(newa)
    
    return nexta
  
def getvectorfortime(b,a):
    newb=b.values.tolist()
    c=[0]*len(a)
    for i in range (len(newb)):
        am= newb[i][1]
        
        for j in range (len(am)):
            c[a.index(am[j])]+=1
    return c
        

keywords=getkeywords()
data=getDATA()
labels=getLabels()

def create_event_timesteps(start_time, step_size, end_time, length, incident_type='Brand'):
  """
  
  step_size has to be a pd.timeDelta to work, otherwise it will fail
  
  Returns the timeseries and the labels for the timeseries
  
  
  """
  
  events = []
  
  current_time = start_time + 0*step_size
  
  while current_time < end_time:
    mask = (labels['start_incident'] >= (current_time)) & (labels['start_incident'] <= current_time+step_size) & (labels['mcl'] == incident_type)
    tmp = labels[mask]
    if len(tmp) > 0:
      events.append(current_time)
    current_time += step_size
    
  
  dataset = []
  labels = []
  for event in events:
    for j in range(length+2):
      timeseries = []
      start_time = event - (j + 1)*step_size
      for i in range(length):
        mask = (data['created_at'] >= start_time+i*step_size) & (data['created_at'] <= start_time+(i+1)*step_size)
        sub_data = data[mask]
        timeseries.append(getvectorfortime(sub_data,keywords))

      timeseries = np.asarray(timeseries)
      end_time = start_time+length*step_size
      # 1min, 5min, 10min, 15min, 30min
      time_periods = [pd.Timedelta("1 minute"), pd.Timedelta("5 minute"), pd.Timedelta("10 minute"), pd.Timedelta("15 minute"), pd.Timedelta("30 minute")]
      time_label = []
      for period in time_periods:
        label_mask = (labels['start_incident'] >= (end_time)) & (labels['start_incident'] <= (end_time+period)) & (labels['mcl'] == incident_type)
        if len(labels[label_mask]) > 0:
          time_label.append(1)
        else:
          time_label.append(0)
    dataset.append(timeseries)
    labels.append(time_label)

  dataset = np.asarray(dataset)
  labels = np.asarray(labels)
    
  return dataset, labels

def create_timestep(start_time, step_size, length, incident_type='Brand'):
  """
  
  step_size has to be a pd.timeDelta to work, otherwise it will fail
  
  Returns the timeseries and the labels for the timeseries
  
  """
  timeseries = []
  for i in range(length):
    mask = (data['created_at'] >= start_time+i*step_size) & (data['created_at'] <= start_time+(i+1)*step_size)
    sub_data = data[mask]
    timeseries.append(getvectorfortime(sub_data,keywords))
  
  timeseries = np.asarray(timeseries)
  end_time = start_time+length*step_size
  # 1min, 5min, 10min, 15min, 30min
  time_periods = [pd.Timedelta("1 minute"), pd.Timedelta("5 minute"), pd.Timedelta("10 minute"), pd.Timedelta("15 minute"), pd.Timedelta("30 minute")]
  time_label = []
  for period in time_periods:
    label_mask = (labels['start_incident'] >= (end_time)) & (labels['start_incident'] <= (end_time+period)) & (labels['mcl'] == incident_type)
    if len(labels[label_mask]) > 0:
      time_label.append(1)
    else:
      time_label.append(0)
    
  return timeseries, time_label
  


class TweetSeqGenerator(Sequence):

    def __init__(self, data, labels, batch_size, normalize=False):
        self.data = data
        self.labels = labels
        if len(self.data) != len(self.labels):
            raise ValueError("Data and Labels have to be same size")
        self.batch_size = batch_size
        self.normalize = normalize

        # If normalize, then normalize each timeseries to sum to 1 on each keyword
        if self.normalize:
            for i in range(len(self.data)):
                # Normalizes the rows of the matrix to 1
                # Axis=0 should normalize the columns to 1
                self.data[i] = self.data[i]/np.max(self.data[i], axis=1)

    def __getitem__(self, index):
        """
        Go through each set of files and augment them as needed
        :param index:
        :return:
        """
        timeseries = self.data[index * self.batch_size:(index + 1) * self.batch_size]
        labels = self.labels[index * self.batch_size:(index + 1) * self.batch_size]

        return timeseries, labels

    def __len__(self):
        """
        Returns the length of the list of paths, as the number of events is not known
        :return:
        """
        return int(np.ceil(len(self.data) / float(self.batch_size)))

    def on_epoch_end(self):
        self.data, self.labels = shuffle(self.data, self.labels)


def generate_timeseries(category, length, limit=None):
    """
    Generates timeseries from the tweets, and gets the next event to occur in the future
    """
    return NotImplementedError


def clean_tweet(tweet):
    """
    Cleans a tweet to just English characters, removing URLs, etc.
    What is given is just the tweet itself, not the metadata
    """
    tweet = re.sub(r"http\S+", "", tweet) # removes URLs
    tweet = re.sub(r"[^a-zA-Z0-9]+", ' ', tweet) # Removes non-alphanumeric chars
    tweet = tweet.lower() # Lowercases it

    return tweet


step_size = pd.Timedelta("1 minute")
start_time = pd.to_datetime("2019-04-26 00:00:00")
end_time = pd.to_datetime("2019-04-29 00:00:00")
length = 30

tweet_keywords, melding = create_event_timesteps(start_time, step_size, end_time, length=length)

#tweet_keywords = []
#melding = []
#for i in range(2800):
#  timeseries, label = create_timestep(start_time, step_size, length=length)
#  tweet_keywords.append(timeseries)
#  melding.append(label)
#  start_time += step_size * (length/30)




    SELECT created_at,keywords FROM `hdc-politie-team-3.politie.tweets` WHERE iso_language_code='nl' and created_at BETWEEN "2019-04-26" AND "2019-04-28" ORDER BY created_at ASC

    


  SELECT * FROM `hdc-politie-team-3.politie.meldkamer` WHERE start_incident BETWEEN "2019-04-26" AND "2019-04-28" ORDER BY start_incident ASC

  


In [6]:
maxlen = length
sequences = tweet_keywords
next_melding = melding

x = np.asarray(sequences)
y = np.asarray(next_melding)

x, y = shuffle(x,y)

validation_cut = int(0.25*len(x))
x1 = x[:validation_cut]
y1 = y[:validation_cut]

x = x[validation_cut:]
y = y[validation_cut:]

print("X Hsape: {}".format(x.shape))
print("Y Shape: {}".format(y.shape))

X Hsape: (2100, 30, 1100)
Y Shape: (2100, 5)


In [ ]:
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(maxlen, len(tweet_keywords[0][0]))))
model.add(LSTM(128))
model.add(Dense(len(melding[0])))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

train_gen = TweetSeqGenerator(data=x, labels=y, batch_size=32)
val_gen = TweetSeqGenerator(data=x1, labels=y1, batch_size=32)

model.fit_generator(generator=train_gen, validation_data=val_gen, use_multiprocessing=True, workers=4, epochs=1000)

model.save("1000_b32_2layer.h5")

Epoch 1/1000
66/66 [==============================] - 14s 217ms/step - loss: 0.3120 - val_loss: 0.3291
Epoch 2/1000
66/66 [==============================] - 13s 199ms/step - loss: 0.2883 - val_loss: 0.3311
Epoch 3/1000
66/66 [==============================] - 16s 235ms/step - loss: 0.2743 - val_loss: 0.3046
Epoch 4/1000
66/66 [==============================] - 14s 209ms/step - loss: 0.2516 - val_loss: 0.2782
Epoch 5/1000
66/66 [==============================] - 13s 194ms/step - loss: 0.2374 - val_loss: 0.2721
Epoch 6/1000
66/66 [==============================] - 13s 192ms/step - loss: 0.2373 - val_loss: 0.2750
Epoch 7/1000
66/66 [==============================] - 13s 195ms/step - loss: 0.2363 - val_loss: 0.2860
Epoch 8/1000
66/66 [==============================] - 13s 191ms/step - loss: 0.2329 - val_loss: 0.2652
Epoch 9/1000
66/66 [==============================] - 13s 192ms/step - loss: 0.2275 - val_loss: 0.2776
Epoch 10/1000
66/66 [==============================] - 13s 195ms/step - l

Epoch 80/1000
66/66 [==============================] - 13s 199ms/step - loss: 0.2195 - val_loss: 0.2894
Epoch 81/1000
66/66 [==============================] - 13s 198ms/step - loss: 0.2179 - val_loss: 0.2894
Epoch 82/1000
66/66 [==============================] - 13s 199ms/step - loss: 0.2202 - val_loss: 0.2898
Epoch 83/1000
66/66 [==============================] - 13s 199ms/step - loss: 0.2179 - val_loss: 0.2903
Epoch 84/1000
66/66 [==============================] - 13s 200ms/step - loss: 0.2204 - val_loss: 0.2901
Epoch 85/1000
66/66 [==============================] - 13s 201ms/step - loss: 0.2188 - val_loss: 0.2903
Epoch 86/1000
66/66 [==============================] - 13s 200ms/step - loss: 0.2179 - val_loss: 0.2919
Epoch 87/1000
66/66 [==============================] - 13s 198ms/step - loss: 0.2183 - val_loss: 0.2923
Epoch 88/1000
66/66 [==============================] - 13s 198ms/step - loss: 0.2180 - val_loss: 0.2927
Epoch 89/1000
66/66 [==============================] - 13s 197ms